# Hybrid Engagement Package Enrichment Notebook
This notebook is intended to explore the capabilities of the OEA Hybrid Engagement package by processing SIS student data into a single table.

**It is necessary that you review and execute the Microsoft Education Insights module pipeline, before testing this Hybrid Engagement enrichment notebook.**

Below describes the execution process of the notebook:
* First initialize the OEA framework class noptebook.
* Then the notebook processes the Insights module roster data ingested into stage 2, by re-writing the schema to only include student data for mapping of the package Power BI dashboard. The tables used are:
    * stage2p/M365/Person_pseudo,
    * stage2p/M365/PersonOrganizationRole_pseudo,
    * stage2p/M365/Organization_pseudo,
    * stage2p/M365/RefDefinition_pseudo, and
    * stage2np/M365/Person_lookup.
* The final resulting tables are written to stage 3:
    * stage3p/hybrid_engagement/Student_pseudo
    * stage3np/hybrid_engagement/Student_lookup

In [1]:
%run /OEA_py

StatementMeta(, 55, -1, Finished, Available)

2022-09-28 16:42:55,257 - OEA - DEBUG - OEA initialized.
OEA initialized.


In [2]:
# 0) Initialize the OEA framework.
oea = OEA()

StatementMeta(sparkMed, 55, 3, Finished, Available)

2022-09-28 16:43:01,647 - OEA - DEBUG - OEA initialized.
2022-09-28 16:43:01,647 - OEA - DEBUG - OEA initialized.
OEA initialized.


## Data Aggregations

### SIS Student_pseudo table creation

In [12]:
dfInsights_person = oea.load('M365', 'Person_pseudo')
dfInsights_personOrgRole = oea.load('M365', 'PersonOrganizationRole_pseudo')
dfInsights_organization = oea.load('M365', 'Organization_pseudo')
dfInsights_refDefinition = oea.load('M365', 'RefDefinition_pseudo')

StatementMeta(sparkMed, 55, 13, Finished, Available)

In [13]:
dfInsights = dfInsights_personOrgRole.join(dfInsights_person, dfInsights_personOrgRole.PersonId_pseudonym == dfInsights_person.Id_pseudonym, how='inner')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'RefRoleId', 'RefGradeLevelId', 'OrganizationId')
display(dfInsights.limit(10))

StatementMeta(sparkMed, 55, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, df22c110-065b-4b11-8470-d17be644f894)

In [14]:
dfInsights = dfInsights.join(dfInsights_organization, dfInsights.OrganizationId == dfInsights_organization.Id, how='inner')
dfInsights = dfInsights.withColumnRenamed('Name', 'OrganizationName')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'RefRoleId', 'RefGradeLevelId', 'OrganizationId', 'OrganizationName')
display(dfInsights.limit(10))

StatementMeta(sparkMed, 55, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, ca608de9-0ed5-4146-b613-5d788d2318fd)

In [15]:
dfInsights = dfInsights.join(dfInsights_refDefinition, dfInsights.RefRoleId == dfInsights_refDefinition.Id, how='inner')
dfInsights = dfInsights.withColumnRenamed('Code', 'PersonRole')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'RefGradeLevelId', 'OrganizationId', 'OrganizationName')
display(dfInsights.limit(10))

StatementMeta(sparkMed, 55, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, da048311-6cf3-4405-8c16-a85b436da923)

In [16]:
dfInsights = dfInsights.join(dfInsights_refDefinition, dfInsights.RefGradeLevelId == dfInsights_refDefinition.Id, how='inner')
dfInsights = dfInsights.withColumnRenamed('Code', 'StudentGrade')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'StudentGrade', 'OrganizationId', 'OrganizationName')
display(dfInsights.limit(10))

StatementMeta(sparkMed, 55, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7f3e752e-5f43-48c9-8b4b-21e77b47f024)

In [17]:
display(dfInsights)

StatementMeta(sparkMed, 55, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5e9c2ba4-90d9-4e02-ae81-1b7e01e883e6)

### Write SIS Student_pseudo table to Stage 3p

In [19]:
dfInsights.coalesce(1).write.format('delta').mode('overwrite').option('header', True).save(oea.stage3p + '/hybrid_engagement/Student_pseudo')

StatementMeta(sparkMed, 55, 20, Finished, Available)

## Data Aggregations

### SIS Student_lookup table creation

In [20]:
dfInsights_person_np = oea.load('M365', 'Person_lookup', stage=oea.stage2np)
dfInsights_person_np = dfInsights_person_np.withColumnRenamed('Id', 'PersonId').withColumnRenamed('Id_pseudonym', 'PersonId_pseudonym')
dfInsights_person_np = dfInsights_person_np.select('PersonId_pseudonym', 'PersonId', 'Surname', 'GivenName', 'MiddleName')
display(dfInsights_person_np.limit(10))

StatementMeta(sparkMed, 55, 21, Finished, Available)

SynapseWidget(Synapse.DataFrame, ec1ddca4-d87f-4568-8cd5-1338eb2ef657)

### Write SIS Student_lookup table to Stage 3np

In [21]:
dfInsights_person_np.coalesce(1).write.format('delta').mode('overwrite').option('header', True).save(oea.stage3np + '/hybrid_engagement/Student_lookup')

StatementMeta(sparkMed, 55, 22, Finished, Available)